In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
import torch.utils.data as data_utils

In [3]:
from torcheval.metrics import R2Score

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("SOCR-HeightWeight.csv")
df.rename(columns={"Height(Inches)": "X", "Weight(Pounds)": "y"}, inplace=True)
df

,Index,X,y
0,1,65.78331,112.9925
1,2,71.51521,136.4873
2,3,69.39874,153.0269
3,4,68.21660,142.3354
4,5,67.78781,144.2971
...,...,...,...
24995,24996,69.50215,118.0312
24996,24997,64.54826,120.1932
24997,24998,64.69855,118.2655
24998,24999,67.52918,132.2682


In [6]:
torch.Tensor([[i] for i in df['X'].values])

tensor([[65.7833],
        [71.5152],
        [69.3987],
        ...,
        [64.6985],
        [67.5292],
        [68.8776]])

In [7]:
training_data = data_utils.TensorDataset(torch.Tensor([[i] for i in df['X'].values]), torch.Tensor(df['y'].values))

In [8]:
batch_size = 10

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(training_data, batch_size=batch_size)


In [9]:
train_dataloader.dataset

In [10]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        logits = self.linear(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)


In [12]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    predicted = torch.Tensor()
    all_y = torch.Tensor()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            all_y = torch.cat((all_y, y))
            # print(pred)
            predicted = torch.cat((predicted, torch.flatten(pred)))
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    metric = R2Score()
    # print(all_y)
    # print(predicted)
    metric.update(predicted, all_y)
    print('accuracy:', metric.compute())
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 5623.439453  [   10/25000]
loss: 4184.526367  [ 1010/25000]
loss: 3319.743164  [ 2010/25000]
loss: 3099.956299  [ 3010/25000]


/home/amitgaru2/anaconda3/envs/cvf/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 2540.309082  [ 4010/25000]
loss: 1940.781616  [ 5010/25000]
loss: 1366.541504  [ 6010/25000]
loss: 1150.384888  [ 7010/25000]
loss: 773.056274  [ 8010/25000]
loss: 694.160767  [ 9010/25000]
loss: 308.215881  [10010/25000]
loss: 279.358612  [11010/25000]
loss: 152.124939  [12010/25000]
loss: 391.067627  [13010/25000]
loss: 220.764252  [14010/25000]
loss: 274.008667  [15010/25000]
loss: 124.816643  [16010/25000]
loss: 190.230759  [17010/25000]
loss: 100.080429  [18010/25000]
loss: 60.804913  [19010/25000]
loss: 230.611328  [20010/25000]
loss: 208.466873  [21010/25000]
loss: 177.686523  [22010/25000]
loss: 231.261841  [23010/25000]
loss: 206.177658  [24010/25000]
accuracy: tensor(0.2105)
Epoch 2
-------------------------------
loss: 207.572876  [   10/25000]
loss: 129.258438  [ 1010/25000]
loss: 247.230774  [ 2010/25000]
loss: 136.167542  [ 3010/25000]
loss: 131.769775  [ 4010/25000]
loss: 344.206146  [ 5010/25000]
loss: 83.138031  [ 6010/25000]
loss: 137.111618  [ 7010/25000]
loss:

KeyboardInterrupt: 